In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob # To import all the calibration images

In [ ]:
class Lines():
     def __init__(self,first=True,recent_xfitted=[],bestx=None,best_fit=None,current_fit=[np.array([False])],
                 radius_of_curvature = None,
                 line_base_pos = None,diffs = np.array([0,0,0], dtype='float'),allx = None,ally=None):
        # was the line detected in the last iteration?
        self.first = first
        # x values of the last n fits of the line
        self.recent_xfitted = recent_xfitted
        #average x values of the fitted line over the last n iterations
        self.bestx = bestx
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = best_fit
        #polynomial coefficients for the most recent fit
        self.current_fit = current_fit
        #radius of curvature of the line in some units
        self.radius_of_curvature = radius_of_curvature
        #distance in meters of vehicle center from the line
        self.line_base_pos = line_base_pos
        #difference in fit coefficients between last and new fits
        self.diffs = diffs
        #x values for detected line pixels
        self.allx = allx
        #y values for detected line pixels
        self.ally = ally
    def calibrate(objpoints,imgpoints):
        objp = np.zeros((6*9,3), np.float32) 
        objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    # Make a list of calibration images
        images = glob.glob('/Users/abylikhsanov1/AI/carnd/term1/advanced-lane-lines/camera_cal/calibration*.jpg') # Please update if needed
        for fname in images:
            image = cv2.imread(fname)
            gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (9,6), None) # Finding any distorted pixel locations
            if ret is True: # If we found any, we shall append the coordinates and the pixels itself
                objpoints.append(objp)
                imgpoints.append(corners)
                cv2.drawChessboardCorners(image, (9,6), corners, ret)
        cv2.destroyAllWindows()
    def process_image(img):
        %matplotlib inline
    #img = cv2.imread('test_images/straight_lines1.jpg') # Now, let's undistort our first road image
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        fig,(ax1) = plt.subplots(1,1,figsize=(10,5))
        ax1.imshow(img)
    # Calibrating and removing the distortion
        img_size = (img.shape[1],img.shape[0]) # We are making img_size[0] as x axis values and [1] as y axis
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,img_size,None,None) # Calibration, moving imgpoints towards the coordinate
        dst = cv2.undistort(img, mtx, dist, None, mtx)

        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=30)
        ax2.imshow(dst)
        ax2.set_title('Undistorted Image', fontsize=30)
        combined_binary = gradient(dst)
        warped,M = warp(combined_binary)
        result,left_fitx,right_fitx,ploty,left_fit,right_fit = fit1(warped,left_fit,right_fit)
        final = unwarp(result,img,left_fitx,right_fitx,ploty,M)
        count = count + 1
        return final